<a href="https://colab.research.google.com/github/jgranizo/HackNJIT2024/blob/main/Final_touches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
# Additional libraries as needed
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

#regulrization
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# Technical Analysis library

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
merged_df = pd.read_csv('/content/drive/MyDrive/KaggleDatasets/final_df_train.csv')


In [7]:
merged_df.head()

,Tesla,Ford,Tesla_Log_Return,Ford_Log_Return,Tesla_MA_5,Tesla_MA_50,Tesla_Volatility,polarity,date,Tesla_MA_100,...,Tesla_Bollinger_Upper_5,Tesla_Bollinger_Lower_5,Tesla_Bollinger_Upper_50,Tesla_Bollinger_Lower_50,Tesla_RSI_5,Tesla_RSI_50,Tesla_MACD,Tesla_Log_Return_Lag1,Tesla_Log_Return_Lag5,Tesla_Direction
0,21.847334,6.993325,-0.001281,-0.012695,19.543734,16.165733,0.057309,0.0,2019-10-28,15.806087,...,24.420928,14.666539,19.311189,13.020278,98.282345,70.265892,1.152629,0.090695,-0.013518,0
1,21.081333,7.017694,-0.035691,0.003478,20.352267,16.284920,0.060164,0.0,2019-10-29,15.880567,...,24.427467,16.277067,19.708293,12.861547,85.149520,66.523709,1.264520,-0.001281,0.008172,0
2,21.000668,6.936469,-0.003834,-0.011642,21.156667,16.403787,0.060533,0.0,2019-10-30,15.948653,...,22.709471,19.603863,20.058121,12.749452,84.844643,66.464741,1.331339,-0.035691,-0.003528,0
3,20.994667,6.977082,-0.000286,0.005838,21.359867,16.529240,0.060770,0.0,2019-10-31,16.013867,...,22.278184,20.441550,20.373656,12.684824,68.290934,67.700057,1.368039,-0.003834,0.162707,0
4,20.887333,7.220751,-0.005126,0.034328,21.162267,16.650787,0.059949,0.0,2019-11-01,16.083233,...,21.940508,20.384025,20.654334,12.647239,0.000000,67.130176,1.372640,-0.000286,0.090695,0


In [8]:
test_size = int(len(merged_df) * 0.2)  # e.g., 20% of data for testing
train_data = merged_df[:-test_size]
test_data = merged_df[-test_size:]

In [13]:
test_data

,Tesla,Ford,Tesla_Log_Return,Ford_Log_Return,Tesla_MA_5,Tesla_MA_50,Tesla_Volatility,polarity,date,Tesla_MA_100,...,Tesla_Bollinger_Upper_5,Tesla_Bollinger_Lower_5,Tesla_Bollinger_Upper_50,Tesla_Bollinger_Lower_50,Tesla_RSI_5,Tesla_RSI_50,Tesla_MACD,Tesla_Log_Return_Lag1,Tesla_Log_Return_Lag5,Tesla_Direction
808,123.559998,11.513170,0.002755,0.015760,119.691998,163.182000,0.058716,0.295,2023-01-12,212.283034,...,128.183565,111.200432,230.527457,95.836543,93.891113,31.961699,-14.111938,0.036109,-0.029469,1
809,122.400002,10.904507,-0.009432,-0.054315,121.559999,161.073600,0.053664,0.000,2023-01-13,210.607901,...,125.803749,117.316249,226.739179,95.408020,84.592627,31.767554,-13.049695,0.002755,0.024352,0
810,131.490005,10.861642,0.071636,-0.003939,123.904001,159.403800,0.058217,0.103,2023-01-17,208.958267,...,133.174309,114.633693,223.706331,95.101269,86.901797,35.370600,-11.343610,-0.009432,0.057655,1
811,128.779999,10.638751,-0.020825,-0.020734,125.890001,157.673200,0.035347,0.000,2023-01-18,207.275100,...,133.918080,117.861922,220.474509,94.871890,78.098464,34.963334,-10.093844,0.071636,-0.007711,0
812,127.169998,10.441579,-0.012581,-0.018707,126.680000,156.067200,0.034486,0.000,2023-01-19,205.586100,...,134.152553,119.207447,217.768135,94.366264,63.246130,35.737123,-9.128087,-0.020825,0.036109,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,216.520004,10.623935,0.020719,-0.007871,220.676001,248.037200,0.035287,0.649,2023-10-24,253.318000,...,246.192780,195.159222,281.669914,214.404485,9.558976,46.079094,-9.012315,0.000424,0.003656,1
1005,212.419998,10.763846,-0.019118,0.013084,214.624002,247.626400,0.034769,0.860,2023-10-25,253.302500,...,221.832227,207.415777,282.490016,212.762783,11.520854,46.502757,-10.148612,0.020719,-0.048931,0
1006,205.759995,10.586625,-0.031855,-0.016602,211.754001,247.229599,0.034855,0.691,2023-10-26,253.184000,...,219.451301,204.056701,283.538131,210.921068,19.350697,46.613870,-11.454502,-0.019118,-0.097616,0
1007,207.300003,9.290113,0.007457,-0.130641,210.816000,246.991199,0.035994,0.767,2023-10-27,253.043900,...,219.454959,202.177042,284.195856,209.786543,36.066555,47.931418,-12.224250,-0.031855,-0.037588,1


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report

# Define features and target for classification
X_train_classification = test_data.drop(columns=['Tesla_Log_Return', 'Tesla_Direction', 'date'])
y_train_classification = test_data['Tesla_Direction']
X_train_regression = test_data.drop(columns=['Tesla_Direction', 'date', 'Tesla_Log_Return'])
y_train_regression = test_data['Tesla_Log_Return']


In [16]:
import joblib
from xgboost import XGBRegressor

classifier_model = joblib.load('/content/drive/MyDrive/KaggleDatasets/XGBoost_models/best_xgb_classifier.pkl')
regressor_model = XGBRegressor()

regressor_model.load_model('/content/drive/MyDrive/KaggleDatasets/XGBoost_models/best_xgb_regressor.model')

In [18]:
# Convert data to DMatrix format for GPU training
classification_params = {
    'subsample': 0.8,
    'reg_lambda': 10,
    'reg_alpha': 0,
    'n_estimators': 500,
    'min_child_weight': 10,
    'max_depth': 8,
    'learning_rate': 0.005,
    'gamma': 0,
    'colsample_bytree': 0.6,
    'tree_method': 'gpu_hist'  # Enable GPU
}

regression_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.01,
    'max_depth': 5,
    'n_estimators': 200,
    'subsample': 0.8,
    'tree_method': 'gpu_hist'  # Enable GPU
}

# Continue training the classification model on GPU
classifier_model.set_params(**classification_params)
classifier_model.fit(X_train_classification, y_train_classification, xgb_model=classifier_model)

# Continue training the regression model on GPU
regressor_model.set_params(**regression_params)
regressor_model.fit(X_train_regression, y_train_regression, xgb_model=regressor_model)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:01:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:01:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "early_stopping", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:01:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score='2.139371E-3', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [19]:
joblib.dump(classifier_model, '/content/drive/MyDrive/KaggleDatasets/XGBoost_models/test_models_new/best_xgb_classifier.pkl')
regressor_model.save_model('/content/drive/MyDrive/KaggleDatasets/XGBoost_models/test_models_new/best_xgb_regressor.model')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:05:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:05:45] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
